In [1]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle

In [2]:
Rsphere = 0.34
Espread = 4.0e-3
npt = 4096*2
nturn = 5

# Read MLI input file

In [3]:
elems0,lattices,labor = mli.readInputfile('mli.in.t3_iota_8_4_t0p4')
del(lattices[-1])

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [4]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [5]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)
elems[2].driftexact = 1

In [6]:
labor = ['iotaline','mapout','fin']

# thick sext 2 thin multipole

In [7]:
elemList,latticeList=mli.sext2thin(elems,lattices,brho=0.50204778184582999)

In [8]:
mli.writeInputfile(elemList,latticeList,labor)

In [9]:
mli.run()

In [10]:
M,G=mli.readTransferMap()

In [11]:
M

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [12]:
G.head()

,exponents,GP
28,f( 30 00 00 ),-23.555531
29,f( 21 00 00 ),-62.881728
33,f( 20 00 01 ),21.191487
34,f( 12 00 00 ),-51.643332
38,f( 11 00 01 ),54.729188


In [13]:
indexThin = []
k2lList = []
for i,item in enumerate(elemList):
    if 'thlm' == item.elem:
        indexThin.append(i)
        k2lList.append(item.k2l)
nThin = len(indexThin)
print(nThin)

18


# Optimize

### Build objective

In [14]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
print(k)

1.4544633270832747


In [15]:
emit = 3.3e-6
bg = gam0*bet0
sx = 0.5*np.sqrt(betx*emit)
spx = 0.5*np.sqrt((1+alfx*alfx)/betx*emit)
sy = 0.5*np.sqrt(betx*emit)
spy = 0.5*np.sqrt((1+alfx*alfx)/betx*emit)
se = 0.5*2.0e-3*bet0
print(sx,spx,se)

0.001249568486716045 0.0011232468237651846 0.0009999942366540901


In [16]:
def getWeight(g):
    w = np.zeros(len(g))
    for i,item in enumerate(g['exponents']):
        nx = int(item[3])
        npx = int(item[4])
        ny = int(item[6])
        npy = int(item[7])
        nE = int(item[10])
        w[i] = np.power(sx,nx)*np.power(spx,npx)*np.power(sy,ny)*np.power(spy,npy)*np.power(se,nE)
    return w

In [17]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
W2 = getWeight(G2_ref)
G2_ref

,exponents,GP
28,f( 30 00 00 ),0.000000
29,f( 21 00 00 ),0.000000
33,f( 20 00 01 ),0.727236
34,f( 12 00 00 ),0.000000
38,f( 11 00 01 ),0.000000
39,f( 10 20 00 ),0.000000
40,f( 10 11 00 ),0.000000
43,f( 10 02 00 ),0.000000
48,f( 10 00 02 ),0.000000
49,f( 03 00 00 ),0.000000


In [18]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2
W3 = getWeight(G3_ref)
G3_ref

,exponents,GP
84,f( 40 00 00 ),0.000000
85,f( 31 00 00 ),0.000000
89,f( 30 00 01 ),0.000000
90,f( 22 00 00 ),0.000000
94,f( 21 00 01 ),0.000000
95,f( 20 20 00 ),0.000000
96,f( 20 11 00 ),0.000000
99,f( 20 02 00 ),0.000000
104,f( 20 00 02 ),0.000004
105,f( 13 00 00 ),0.000000


In [19]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0
W4 = getWeight(G4_ref)

In [20]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0
W5 = getWeight(G5_ref)

In [21]:
elemListTrack = elemList[:3]
raysin  =mli.getElem.raytrace(file1='rays.in',type='readonly')
dump    =mli.getElem.particledump(file='rays.out',precision=15)
track   =mli.getElem.autotrack(type='symplectic',order=5)
fin     =mli.getElem.fin
nlinsert = mli.getElem.nlinsert(steps=100)
readmap = mli.getElem.tmi(name='readmap')
savemap = mli.getElem.stm(name='savemap')
getmap    = mli.getElem.gtm(name='getmap')
elemListTrack = elemListTrack + [raysin,dump,track,fin,nlinsert,readmap,savemap,getmap]
trackLine = [mli.getElem.line(name='trackLine',elemList = ['nlinsert','getmap',dump.name])]
trackLabor = mli.buildLabor(['readmap','savemap','clear',raysin.name,track.name,str(nturn)+'*'+trackLine[0].name,'fin'])
mli.writeInputfile(elemListTrack,trackLine,trackLabor)

### prepare particles

In [22]:
# def MLI2norm(data_in,sign=1):
#     data=data_in.copy()
#     data[:,5] = np.sqrt(1.0-2.0*data[:,5]/bet0+data[:,5]**2)-1.0
#     data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
#     data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
#     data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
#     data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
#     return data
    
# def norm2MLI(data_in,sign=1):
#     data=data_in.copy()
#     data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
#     data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
#     data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
#     data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
#     data[:,5] = -np.sqrt((1.0+data[:,5])**2+1.0/(bet0*gam0)**2)+1.0/bet0
#     return data

In [23]:
# def getTBT(npt,nturn,fname='rays.out'):
#     TBT = np.loadtxt(fname)
#     TBT = TBT[:npt*nturn,:6]
#     out = np.zeros([npt,nturn,6])
#     for i in range(nturn):
#         out[:,i,:] = TBT[i*npt:(i+1)*npt,:]
#         out[:,i,:] = MLI2norm(out[:,i,:])
#     return out,TBT[npt*(nturn-1):,:]

In [24]:
# def getInv(xn,pxn,yn,pyn,delta,tau=NL_t):
#     z = xn + 1j*yn
#     U = np.real(z/np.sqrt(1-z**2)*np.arcsin(z))
#     W = np.real(2*xn/np.sqrt(1-z**2)*np.arcsin(z))
#     Hn = 0.5*(xn**2+pxn**2+yn**2+pyn**2)   +tau*U/(1.0+delta)
#     In = (xn*pyn -yn*pxn)**2 +xn**2+pxn**2 +tau*W/(1.0+delta)
#     return Hn,In

# def getInvTBT(TBT):
#     npt,nturn,dummy = TBT.shape
#     InvTBT = np.zeros([npt,nturn,2])
#     for iturn in range(nturn):
#         data = TBT[:,iturn,:]
#         for ipt in range(npt):
#             xn    = data[ipt,0]
#             pxn   = data[ipt,1]
#             yn    = data[ipt,2]
#             pyn   = data[ipt,3]
#             delta = data[ipt,5]
#             InvTBT[ipt,iturn,:] = getInv(xn,pxn,yn,pyn,delta)
#     return InvTBT

In [25]:
# def getINVobj(INV,INV0):
#     obj=0.0
#     tmpHmax = 0.0
#     tmpImax = 0.0
#     for i in range(nturn):
#         tmpH = (INV[:,i,0]/INV0[:,0,0]-1.0)**2 
#         tmpI = (INV[:,i,1]/INV0[:,0,1]-1.0)**2
#         tmpHmax = np.max([np.max(tmpH),tmpHmax])
#         tmpImax = np.max([np.max(tmpI),tmpImax])
#         tmp = np.sum(tmpH) + np.sum(tmpI)
#         obj = obj + tmp
#         if obj > 10:
#             return 200*np.sqrt(obj)/(npt*2)/nturn, np.max([tmpHmax,tmpImax])*15
#         obj = obj + tmp
#     return 200*np.sqrt(obj)/(npt*2)/nturn, np.max([tmpHmax,tmpImax])*15

In [26]:
# pData=np.zeros([npt,6])
# x = np.random.random([npt,4])
# c0 = np.cos(x[:,0])
# s0 = np.sin(x[:,0])
# c1 = np.cos(x[:,1])
# s1 = np.sin(x[:,1])
# c2 = np.cos(x[:,2])
# s2 = np.sin(x[:,2])
# pData[:,0] = Rsphere*c0
# pData[:,1] = Rsphere*s0*c1
# pData[:,2] = Rsphere*s0*s1*c2
# pData[:,3] = Rsphere*s0*s1*s2
# pData[:,5] = (x[:,3]-0.5)*2*Espread
# pDataIn = norm2MLI(pData,sign=1)

In [27]:
#%%
def objFunc(arg): 
    target = opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = opt.id_generator()
    shutil.copytree('origin', target)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(indexThin):
        elemList[j]['k2l']=arg[i]
        
    mli.writeInputfile(elemList,latticeList,labor)
    mli.run()
    M,G = mli.readTransferMap()
    
    test = np.all(G2_ref['exponents'].values == G.loc[28 :76 ]['exponents'].values) and \
           np.all(G3_ref['exponents'].values == G.loc[84 :200]['exponents'].values) and \
           np.all(G4_ref['exponents'].values == G.loc[210:450]['exponents'].values) and \
           np.all(G5_ref['exponents'].values == G.loc[462:910]['exponents'].values)
    
    if not test:
        raise ValueError("PROBLEM!!!")
        
    obj = np.sum(((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)*W2)**2) + \
          np.sum(((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)*W3)**2) + \
          np.sum( (G.loc[210:450,'GP'].values*W4)**2) + \
          np.sum( (G.loc[462:910,'GP'].values*W5)**2)
    
    os.chdir('..')
    shutil.rmtree(target)
    return obj

In [28]:
objFunc([0]*nThin)

3.054941925423305e-14

In [29]:
# #%% run optim
bounds = [(-100,100)]*nThin
result=opt.differential_evolution(objFunc, bounds, ncore=32, popsize=32*8,
                                  disp=True, polish=True, maxtime=60*10) 
# #                                     stop running at maximum 10 min

differential_evolution step 1: f(x)= 1.71295e-13
differential_evolution step 1: f(x)= 1.71295e-13
differential_evolution step 2: f(x)= 1.71295e-13
differential_evolution step 3: f(x)= 1.71295e-13
differential_evolution step 4: f(x)= 1.71295e-13
differential_evolution step 5: f(x)= 1.71295e-13
differential_evolution step 6: f(x)= 1.71295e-13
differential_evolution step 7: f(x)= 1.61378e-13
differential_evolution step 8: f(x)= 1.16718e-13
differential_evolution step 9: f(x)= 1.16718e-13
differential_evolution step 10: f(x)= 1.16718e-13
differential_evolution step 11: f(x)= 1.16718e-13
differential_evolution step 12: f(x)= 1.16718e-13
differential_evolution step 13: f(x)= 1.16718e-13
differential_evolution step 14: f(x)= 1.16718e-13
differential_evolution step 15: f(x)= 1.16718e-13
differential_evolution step 16: f(x)= 1.16718e-13
differential_evolution step 17: f(x)= 1.16718e-13
differential_evolution step 18: f(x)= 1.16718e-13
differential_evolution step 19: f(x)= 1.16718e-13
differenti

In [30]:
with open('result.thin.0.5sig.sext','wb') as fp:
    pickle.dump(result,fp)

In [31]:
with open('result.thin.0.5sig.sext','rb') as fp:
    result=pickle.load(fp)

In [32]:
result.message

'Maximum time has been exceeded.'

In [33]:
np.sort(result.population_energies)[:16]

array([1.16717923e-13, 1.16717923e-13, 1.23017648e-13, 1.27450274e-13,
       1.31427861e-13, 1.50897967e-13, 1.51074635e-13, 1.52100334e-13,
       1.58671092e-13, 1.61378328e-13, 1.66310686e-13, 1.73809436e-13,
       1.75657187e-13, 1.84476133e-13, 1.96939249e-13, 1.98088406e-13])

In [34]:
np.sort(result.population_energies)[-16:]

array([8.12477612e-13, 8.23903123e-13, 8.27011418e-13, 8.55323321e-13,
       8.66799971e-13, 8.69954517e-13, 8.72616859e-13, 8.85579873e-13,
       8.87715208e-13, 8.89097183e-13, 9.00754172e-13, 9.11447213e-13,
       9.23687527e-13, 9.53083746e-13, 9.98130142e-13, 1.14760812e-12])

In [35]:
# pop = copy(result.population[0]-0.5)
# pop[:nThin] = pop[:nThin]*200
# pop

In [ ]:
while True:
    previous_result = result
    if hasattr(result,'x'): 
        break
    result = opt.differential_evolution(objFunc, bounds, ncore=32, 
                                           prev_result=previous_result, 
                                           disp=True, maxtime=60*10)
    with open('result.thin.0.5sig.sext','wb') as fp:
        pickle.dump(result,fp)
        time.sleep(0.1)

differential_evolution step 0: f(x)= 1.16718e-13
differential_evolution step 0: f(x)= 9.27319e-14
differential_evolution step 1: f(x)= 9.27319e-14
differential_evolution step 2: f(x)= 9.27319e-14
differential_evolution step 3: f(x)= 9.27319e-14
differential_evolution step 4: f(x)= 9.27319e-14
differential_evolution step 5: f(x)= 9.27319e-14
differential_evolution step 6: f(x)= 9.27319e-14
differential_evolution step 7: f(x)= 9.27319e-14
differential_evolution step 8: f(x)= 9.27319e-14
differential_evolution step 9: f(x)= 9.27319e-14
differential_evolution step 10: f(x)= 7.66609e-14
differential_evolution step 11: f(x)= 7.66609e-14
differential_evolution step 12: f(x)= 7.66609e-14
differential_evolution step 13: f(x)= 7.66609e-14
differential_evolution step 14: f(x)= 7.66609e-14
differential_evolution step 15: f(x)= 7.66609e-14
differential_evolution step 16: f(x)= 7.66609e-14
differential_evolution step 17: f(x)= 7.66609e-14
differential_evolution step 18: f(x)= 7.66609e-14
differentia

differential_evolution step 11: f(x)= 2.59038e-14
differential_evolution step 12: f(x)= 2.59038e-14
differential_evolution step 13: f(x)= 2.59038e-14
differential_evolution step 14: f(x)= 2.59038e-14
differential_evolution step 15: f(x)= 2.59038e-14
differential_evolution step 16: f(x)= 2.59038e-14
differential_evolution step 17: f(x)= 2.59038e-14
differential_evolution step 18: f(x)= 2.59038e-14
differential_evolution step 19: f(x)= 2.59038e-14
differential_evolution step 20: f(x)= 2.59038e-14
differential_evolution step 0: f(x)= 2.59038e-14
differential_evolution step 0: f(x)= 2.59038e-14
differential_evolution step 1: f(x)= 2.59038e-14
differential_evolution step 2: f(x)= 2.59038e-14
differential_evolution step 3: f(x)= 2.59038e-14
differential_evolution step 4: f(x)= 2.59038e-14
differential_evolution step 5: f(x)= 2.01002e-14
differential_evolution step 6: f(x)= 2.01002e-14
differential_evolution step 7: f(x)= 2.01002e-14
differential_evolution step 8: f(x)= 1.9154e-14
differentia

differential_evolution step 2: f(x)= 1.31891e-14
differential_evolution step 3: f(x)= 1.31891e-14
differential_evolution step 4: f(x)= 1.31891e-14
differential_evolution step 5: f(x)= 1.31891e-14
differential_evolution step 6: f(x)= 1.31891e-14
differential_evolution step 7: f(x)= 1.31891e-14
differential_evolution step 8: f(x)= 1.31891e-14
differential_evolution step 9: f(x)= 1.31891e-14
differential_evolution step 10: f(x)= 1.31891e-14
differential_evolution step 11: f(x)= 1.31891e-14
differential_evolution step 12: f(x)= 1.31891e-14
differential_evolution step 13: f(x)= 1.31891e-14
differential_evolution step 14: f(x)= 1.31891e-14
differential_evolution step 15: f(x)= 1.31891e-14
differential_evolution step 16: f(x)= 1.31891e-14
differential_evolution step 17: f(x)= 1.31891e-14
differential_evolution step 18: f(x)= 1.31891e-14
differential_evolution step 19: f(x)= 1.31891e-14
differential_evolution step 20: f(x)= 1.31891e-14
differential_evolution step 0: f(x)= 1.31891e-14
different

differential_evolution step 14: f(x)= 7.73604e-15
differential_evolution step 15: f(x)= 7.73604e-15
differential_evolution step 16: f(x)= 7.73604e-15
differential_evolution step 17: f(x)= 7.73604e-15
differential_evolution step 18: f(x)= 7.73604e-15
differential_evolution step 19: f(x)= 7.73604e-15
differential_evolution step 20: f(x)= 7.73604e-15
differential_evolution step 0: f(x)= 7.73604e-15
differential_evolution step 0: f(x)= 7.73604e-15
differential_evolution step 1: f(x)= 7.73604e-15
differential_evolution step 2: f(x)= 7.73604e-15
differential_evolution step 3: f(x)= 7.73604e-15
differential_evolution step 4: f(x)= 7.73604e-15
differential_evolution step 5: f(x)= 7.73604e-15
differential_evolution step 6: f(x)= 7.73604e-15
differential_evolution step 7: f(x)= 7.73604e-15
differential_evolution step 8: f(x)= 7.73604e-15
differential_evolution step 9: f(x)= 7.73604e-15
differential_evolution step 10: f(x)= 7.73604e-15
differential_evolution step 11: f(x)= 7.73604e-15
differentia

differential_evolution step 4: f(x)= 4.44987e-15
differential_evolution step 5: f(x)= 4.24442e-15
differential_evolution step 6: f(x)= 4.24442e-15
differential_evolution step 7: f(x)= 4.24442e-15
differential_evolution step 8: f(x)= 4.24442e-15
differential_evolution step 9: f(x)= 4.24442e-15
differential_evolution step 10: f(x)= 4.24442e-15
differential_evolution step 11: f(x)= 4.24442e-15
differential_evolution step 12: f(x)= 4.24442e-15
differential_evolution step 13: f(x)= 4.24442e-15
differential_evolution step 14: f(x)= 4.24442e-15
differential_evolution step 15: f(x)= 4.24442e-15
differential_evolution step 16: f(x)= 4.24442e-15
differential_evolution step 17: f(x)= 4.24442e-15
differential_evolution step 18: f(x)= 4.24442e-15
differential_evolution step 19: f(x)= 4.24442e-15
differential_evolution step 20: f(x)= 4.24442e-15
differential_evolution step 0: f(x)= 4.24442e-15
differential_evolution step 0: f(x)= 4.24442e-15
differential_evolution step 1: f(x)= 4.24442e-15
different

In [ ]:
# result.x = copy(result.population[0]-0.5)
# result.x[:nThin] = result.x[:nThin]*200
# result.x

In [ ]:
result